In [ ]:
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

In [ ]:
# load in the file
# This is a way to turn on chunking and lazy evaluation. Opening with mfdataset, or
# setting the chunking in the open_dataset would also achieve this.
ds = xr.tutorial.open_dataset("ROMS_example.nc", chunks={"ocean_time": 1})

In [ ]:
ds

Some entities:
- C(x, y, z, t) - a traces - scalar quantity, i.e. temperature, salinity, nutrient concentration.
- h(x, y) - depth of sea floor below mean sea level.
- f(x, y) - Coriolis parameter.
- u, v, w - components of vector velocity
- $\rho$(x, y, z, t) - density.
- $\zeta$(x, y, t) (zeta) - surface elevation.
- s_rho - $\sigma$(x, y, z) (sigma) is a terrain-following depth (derived from z) coordinate.
It is a stretched vertical coordinate system which essentially “flattens out” the variable bottom at z = −h(x, y).
Vtransform defines a stretched vertical coordinate system.
- $\Omega$ - the vertical velocity in $\sigma$ coordinates.
- xi_rho - $\xi$(x, y) and eta_rho - $\eta$(x, y) are horizontal curvilinear coordinates.

![arakawa_c_grid](pictures/Arakawa_C_grid.png)

In the horizontal ($\xi$, $\eta$), a traditional, centered, second-order finite-difference approximation is
adopted.

![vertical_grid](pictures/Placement_of_variables_on_staggered_vertical_grid.png)

The vertical discretization also uses a second-order finite-difference approximation.

![masked_grid](pictures/Masked_region_within_the_domain.png)

$\psi$ (psi) points are the masked points?

In [ ]:
if ds.Vtransform == 1:
    Zo_rho = ds.hc * (ds.s_rho - ds.Cs_r) + ds.Cs_r * ds.h
    z_rho = Zo_rho + ds.zeta * (1 + Zo_rho / ds.h)
elif ds.Vtransform == 2:
    Zo_rho = (ds.hc * ds.s_rho + ds.Cs_r * ds.h) / (ds.hc + ds.h)
    z_rho = ds.zeta + (ds.zeta + ds.h) * Zo_rho

ds.coords["z_rho"] = z_rho.transpose()  # type: ignore
ds.salt

In [ ]:
ds.salt.isel(xi_rho=50, ocean_time=0).plot()

In [ ]:
section = ds.salt.isel(xi_rho=50, eta_rho=slice(0, 167), ocean_time=0)
section.plot(x="lon_rho", y="z_rho", figsize=(15, 6), clim=(25, 35))
plt.ylim([-100, 1]);

In [ ]:
ds.salt.isel(s_rho=-1, ocean_time=0).plot(x="lon_rho", y="lat_rho")

In [ ]:
proj = ccrs.LambertConformal(central_longitude=-92, central_latitude=29)
fig = plt.figure(figsize=(15, 5))
ax = plt.axes(projection=proj)
ds.salt.isel(s_rho=-1, ocean_time=0).plot(
    x="lon_rho", y="lat_rho", transform=ccrs.PlateCarree()
)

coast_10m = cfeature.NaturalEarthFeature(
    "physical", "land", "10m", edgecolor="k", facecolor="0.8"
)
ax.add_feature(coast_10m)  # type: ignore